In [49]:
import cv2
import numpy as np

In [50]:
# Zhang Suen 细化算法，进行骨架提取
def Zhang_Suen_thinning(img):
    # Get image shape
    H, W = img.shape

    # Prepare output image
    out = np.zeros((H, W), dtype=np.int32)
    out[img[..., 0] > 0] = 1

    # Inverse image (white background, black character)
    out = 1 - out

    while True:
        s1 = []
        s2 = []

        # Step 1 (raster scan)
        for y in range(1, H - 1):
            for x in range(1, W - 1):

                # Skip non-edge pixels
                if out[y, x] > 0:
                    continue

                # Condition 2
                f1 = 0
                if (out[y - 1, x + 1] - out[y - 1, x]) == 1: f1 += 1
                if (out[y, x + 1] - out[y - 1, x + 1]) == 1: f1 += 1
                if (out[y + 1, x + 1] - out[y, x + 1]) == 1: f1 += 1
                if (out[y + 1, x] - out[y + 1, x + 1]) == 1: f1 += 1
                if (out[y + 1, x - 1] - out[y + 1, x]) == 1: f1 += 1
                if (out[y, x - 1] - out[y + 1, x - 1]) == 1: f1 += 1
                if (out[y - 1, x - 1] - out[y, x - 1]) == 1: f1 += 1
                if (out[y - 1, x] - out[y - 1, x - 1]) == 1: f1 += 1

                if f1 != 1:
                    continue

                # Condition 3
                f2 = np.sum(out[y - 1:y + 2, x - 1:x + 2])
                if f2 < 2 or f2 > 6:
                    continue

                # Condition 4 and 5
                if (out[y - 1, x] + out[y, x + 1] + out[y + 1, x]) < 1: continue
                if (out[y, x + 1] + out[y + 1, x] + out[y, x - 1]) < 1: continue

                s1.append([y, x])

        for v in s1:
            out[v[0], v[1]] = 1

        # Step 2 (raster scan)
        for y in range(1, H - 1):
            for x in range(1, W - 1):

                if out[y, x] > 0:
                    continue

                # Same conditions as step 1
                f1 = 0
                if (out[y - 1, x + 1] - out[y - 1, x]) == 1: f1 += 1
                if (out[y, x + 1] - out[y - 1, x + 1]) == 1: f1 += 1
                if (out[y + 1, x + 1] - out[y, x + 1]) == 1: f1 += 1
                if (out[y + 1, x] - out[y + 1, x + 1]) == 1: f1 += 1
                if (out[y + 1, x - 1] - out[y + 1, x]) == 1: f1 += 1
                if (out[y, x - 1] - out[y + 1, x - 1]) == 1: f1 += 1
                if (out[y - 1, x - 1] - out[y, x - 1]) == 1: f1 += 1
                if (out[y - 1, x] - out[y - 1, x - 1]) == 1: f1 += 1

                if f1 != 1:
                    continue

                f2 = np.sum(out[y - 1:y + 2, x - 1:x + 2])
                if f2 < 2 or f2 > 6:
                    continue

                if (out[y - 1, x] + out[y, x + 1] + out[y, x - 1]) < 1: continue
                if (out[y - 1, x] + out[y + 1, x] + out[y, x - 1]) < 1: continue

                s2.append([y, x])

        for v in s2:
            out[v[0], v[1]] = 1

        if len(s1) < 1 and len(s2) < 1:
            break

    out = apply_template_removal(out, H, W)

    out = 1 - out
    out = out.astype(np.uint8) * 255

    return out

In [51]:
# 空洞或噪声消除模板
def apply_template_removal(out, H, W):
    for y in range(1, H-1):
        for x in range(1, W-1):
            P = out[y-1:y+2, x-1:x+2].flatten()
            if (P[1] * P[7] == 1 and sum([P[3], P[4], P[5], P[8]]) == 0) or \
               (P[5] * P[7] == 1 and sum([P[1], P[2], P[3], P[6]]) == 0) or \
               (P[1] * P[3] == 1 and sum([P[2], P[5], P[6], P[7]]) == 0) or \
               (P[3] * P[5] == 1 and sum([P[1], P[4], P[7], P[8]]) == 0) or \
               (sum([P[2], P[4], P[6], P[8]]) == 0 and sum([P[1], P[3], P[5], P[7]]) == 3):
                out[y, x] = 1
    return out

def apply_template(image, template):
    """根据给定模板进行处理，消除噪声或孔洞"""
    h, w = image.shape
    processed_image = image.copy()
    for i in range(1, h - 1):
        for j in range(1, w - 1):
            # 获取8邻域
            neighborhood = image[i - 1:i + 2, j - 1:j + 2]
            # 与模板匹配
            if np.array_equal(neighborhood, template):
                processed_image[i, j] = 0  # 将符合条件的前景点更改为背景点
    return processed_image

# 凹凸点和孤立点模板
templates_bumps_isolated = [
    np.array([[0, 0, 0], [0, 1, 0], [1, 1, 1]]),
    np.array([[1, 0, 0], [1, 1, 0], [1, 0, 0]]),

]
# 孔洞消除模板
templates_holes = [
    np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]]),

]

# 应用模板处理图像
def preprocess_image(image):
    # 处理凹凸点和孤立点
    for template in templates_bumps_isolated:
        image = apply_template(image, template)

    # 处理孔洞
    for template in templates_holes:
        image = apply_template(image, template)

    return image

def preprocess_images(image1, image2):
    """
    处理两个输入图像，分别处理凹凸点、孤立点和孔洞。
    """
    # 分别处理 image1 和 image2
    processed_image1 = preprocess_image(image1)
    processed_image2 = preprocess_image(image2)

    return processed_image1, processed_image2

#可以输出细化后的骨架，先细化在进行归一化

In [52]:
def binarize_image(image, threshold=127):
    """ 将灰度图像二值化 """
    _, binary_image = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)
    return binary_image

def calculate_center_of_mass_from_contour(image):
    """ 通过轮廓提取计算图像的重心坐标 """
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return image.shape[1] // 2, image.shape[0] // 2  # 返回图像中心
    
    max_contour = max(contours, key=cv2.contourArea)
    M = cv2.moments(max_contour)
    
    if M['m00'] == 0:
        return image.shape[1] // 2, image.shape[0] // 2  # 返回图像中心

    center_x = int(M['m10'] / M['m00'])
    center_y = int(M['m01'] / M['m00'])
    
    return center_x, center_y

def resize_and_center_images_to_equal_size(template, student, output_size=(400, 400)):
    """ 调整模板和临摹图像，使它们的框大小一致并居中 """
    # Step 1: 将灰度图像转换为二值图像
    binary_template = binarize_image(template)
    binary_student = binarize_image(student)
    
    # Step 2: 计算模板和临摹图像的外接矩形 (Bounding Box)
    x_T, y_T, w_T, h_T = cv2.boundingRect(binary_template)
    x_C, y_C, w_C, h_C = cv2.boundingRect(binary_student)

    # Step 3: 选择模板和临摹字中较大的宽高，确保两个图像的框大小相等
    max_width = max(w_T, w_C)
    max_height = max(h_T, h_C)

    # Step 4: 调整大小，使两者的框大小一致
    template_resized = cv2.resize(binary_template[y_T:y_T+h_T, x_T:x_T+w_T], (max_width, max_height))
    student_resized = cv2.resize(binary_student[y_C:y_C+h_C, x_C:x_C+w_C], (max_width, max_height))
    
    # Step 5: 创建400x400的空白背景
    blank_template = np.ones(output_size, dtype=np.uint8) * 255  # 模板字空白图
    blank_student = np.ones(output_size, dtype=np.uint8) * 255   # 临摹字空白图
    
    # Step 6: 计算重心
    template_center_x, template_center_y = calculate_center_of_mass_from_contour(template_resized)
    student_center_x, student_center_y = calculate_center_of_mass_from_contour(student_resized)
    
    # Step 7: 将模板字和临摹字放置在400x400的空白图中，确保居中并且框大小一致
    start_x_T = output_size[1] // 2 - template_resized.shape[1] // 2
    start_y_T = output_size[0] // 2 - template_resized.shape[0] // 2
    blank_template[start_y_T:start_y_T+template_resized.shape[0], start_x_T:start_x_T+template_resized.shape[1]] = template_resized
    
    start_x_C = output_size[1] // 2 - student_resized.shape[1] // 2
    start_y_C = output_size[0] // 2 - student_resized.shape[0] // 2
    blank_student[start_y_C:start_y_C+student_resized.shape[0], start_x_C:start_x_C+student_resized.shape[1]] = student_resized
    
    return blank_template, blank_student

#可以输出归一化后的图片


In [53]:
#骨架相似度评价，用的是像素计算。（也可用坐标计算）
# 计算两个点之间的欧几里得距离
def euclidean_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

# 计算一个点到一组骨架点的最小距离
def point_to_skeleton_distance(point, skeleton_points):
    return min(euclidean_distance(point, sk_point) for sk_point in skeleton_points)

# 计算两个骨架图像之间的总距离
def calculate_total_distance(skeleton1, skeleton2):
    skeleton1_points = np.argwhere(skeleton1 == 255)  # 骨架为白色
    skeleton2_points = np.argwhere(skeleton2 == 255)  # 骨架为白色
    total_distance = sum(point_to_skeleton_distance(point, skeleton2_points) for point in skeleton1_points)
    
    print("骨架相似度计算",total_distance)
    
    return total_distance

In [54]:
# -----------------------------笔画提取--------------------------------
def extract_grid_image(imageData, position):
    height, width = imageData.shape[:2]
    grid_height = height // 3
    grid_width = width // 3
    x, y = position
    start_x = x * grid_width
    start_y = y * grid_height
    end_x = start_x + grid_width
    end_y = start_y + grid_height
    end_x = min(end_x, width)
    end_y = min(end_y, height)
    grid_image = imageData[start_y:end_y, start_x:end_x]
    return grid_image

# 定义函数计算图像的Hu矩
def calculate_hu_moments(image):
    if cv2.countNonZero(image) == 0:
        return np.zeros(7)  # 返回零矩阵避免 nan
    moments = cv2.moments(image)
    hu_moments = cv2.HuMoments(moments).flatten()
    return hu_moments


In [55]:
# 计算笔画相似度
def StrokeExtractionAndSimilarityEvaluation(imageData, reference_image):
    handwriting_hu_moments = calculate_hu_moments(imageData)
    template_hu_moments = calculate_hu_moments(reference_image)

    grid_positions = [(0, 0), (0, 1), (0, 2),
                      (1, 0), (1, 1), (1, 2),
                      (2, 0), (2, 1), (2, 2)]

    grid_weights = [0.1, 0.1, 0.1,
                    0.1, 0.5, 0.1,
                    0.1, 0.1, 0.1]

    handwriting_hu_moments_list = []
    template_hu_moments_list = []

    for position in grid_positions:
        handwriting_grid_image = extract_grid_image(imageData, position)
        template_grid_image = extract_grid_image(reference_image, position)

        handwriting_hu_moments = calculate_hu_moments(handwriting_grid_image)
        template_hu_moments = calculate_hu_moments(template_grid_image)

        handwriting_hu_moments_list.append(handwriting_hu_moments)
        template_hu_moments_list.append(template_hu_moments)

    correlation_coefficients = []
    for handwriting_hu_moments, template_hu_moments in zip(handwriting_hu_moments_list, template_hu_moments_list):
        correlation_coefficient = np.corrcoef(handwriting_hu_moments, template_hu_moments)[0, 1]
        if np.isnan(correlation_coefficient):
            correlation_coefficient = 0  # 避免 nan
        correlation_coefficients.append(correlation_coefficient)

    # 确保 correlation_coefficients 和 grid_weights 都是数值类型列表
    correlation_coefficients = np.array(correlation_coefficients, dtype=float)
    grid_weights = np.array(grid_weights, dtype=float)

    # 计算加权和
    weighted_sum = np.dot(correlation_coefficients, grid_weights)

    # 确保 weighted_sum 是浮点数并乘以常量
    weighted_sum = float(weighted_sum) * 76.924
    weighted_sum = round(weighted_sum, 2)

    # 根据相似度值返回相应的描述
    if weighted_sum < 20:
        description = '笔画极其不相似，与模板差异极大'
    elif weighted_sum < 40:
        description = '笔画很不相似，需要大幅度改进'
    elif weighted_sum < 60:
        description = '笔画不太相似，部分偏离模板'
    elif weighted_sum < 80:
        description = '笔画位置基本相似，但仍有改进空间'
    else:
        description = '笔画位置非常相似，与模板高度一致'

    print('笔画相似度:', weighted_sum,description)
    return weighted_sum, description

In [57]:
def main():
    template_path = r"E:\python files\PycharmProjects\CompleteProject\4.png"
    handwriting_path = r"E:\python files\PycharmProjects\CompleteProject\18.png"
    
    # 读取灰度图像
    template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
    handwriting = cv2.imread(handwriting_path, cv2.IMREAD_GRAYSCALE)
        # 反色处理
    # img1 = cv2.bitwise_not(template)
    # img2 = cv2.bitwise_not(handwriting)

    # if img1 is None or img2 is None:
    #     print("Error loading images.")
    #     return
    #    # 骨架提取
    
    out1 = Zhang_Suen_thinning(template)
    out2 = Zhang_Suen_thinning(handwriting)
    aligned_template, aligned_handwriting = preprocess_images(out1, out2)
 
    # 调整临摹字大小并将它们都显示在400x400的空白图中
    aligned_template1, aligned_handwriting1 = resize_and_center_images_to_equal_size( aligned_template,  aligned_handwriting)

    # 计算骨架相似度
    skeleton_similarity_score = calculate_total_distance(aligned_template1, aligned_handwriting1)
    
    # 计算布局相似度，hu矩九宫格
    weighted_score, description = StrokeExtractionAndSimilarityEvaluation(aligned_template1, aligned_handwriting1)

    # 计算总分
    full_score = skeleton_similarity_score * 0.5 + weighted_score * 0.5
    print("总分：", full_score)

if __name__ == "__main__":
    main()

KeyboardInterrupt: 